In [1]:
import pyspark
from delta import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark = (SparkSession.builder.appName("SparkSample").getOrCreate())
spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 16:18:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/12 16:18:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/12 16:18:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Read in a single file

In [2]:
df = spark.read.text("textfile.txt")

# create a temporary view in spark

In [3]:
df.createOrReplaceTempView("vw_temp_view")

In [4]:
spark.sql("select * from vw_temp_view").show(2)

+--------------------+
|               value|
+--------------------+
|Disclaimer: While...|
|                    |
+--------------------+
only showing top 2 rows



# create a view using SQL that will persist

In [5]:
spark.sql("drop table if exists tbl_view")

DataFrame[]

In [6]:
spark.sql("""create table tbl_view as select * from vw_temp_view""")

chgrp: changing ownership of 'file:///user/hive/warehouse/tbl_view': chown: changing group of '/user/hive/warehouse/tbl_view': Operation not permitted


DataFrame[]

In [7]:
spark.sql("drop view if exists vw_view")

DataFrame[]

In [8]:
spark.sql("""create view vw_view as select * from tbl_view""")

DataFrame[]

# CTE (Common Table Expression) is a better way to write complex queries

In [9]:
spark.sql("""with cte_one as (
    select * from vw_view
    )
    select * from cte_one""").show(2)

+--------------------+
|               value|
+--------------------+
|Disclaimer: While...|
|                    |
+--------------------+
only showing top 2 rows



# metadata of the table

In [10]:
spark.sql("DESCRIBE FORMATTED tbl_view").show(truncate=False)

+----------------------------+----------------------------------------------------------+-------+
|col_name                    |data_type                                                 |comment|
+----------------------------+----------------------------------------------------------+-------+
|value                       |string                                                    |NULL   |
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|Catalog                     |spark_catalog                                             |       |
|Database                    |default                                                   |       |
|Table                       |tbl_view                                                  |       |
|Owner                       |sparkuser                                                 |       |
|Created Time       